# 環境について

これまでに、Azure Machine Learningワークスペースで多くの実験を実行し、場合によっては、実験コードを実行する環境で必要な特定のPythonパッケージを指定する必要がありました。このラボでは、環境をもう少し詳しく見ていきます。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験用のデータを準備する

このラボでは、糖尿病患者の詳細を含むデータセットを使用します。以下のセルを実行して、このデータセットを作成します（以前のラボで作成した場合、コードは新しいバージョンを作成します）

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

#Create a tabular dataset from the path on the datastore (this may take a short while)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# Register the tabular dataset
tab_data_set = tab_data_set.register(workspace=ws, 
                           name='diabetes dataset',
                           description='diabetes data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Dataset ready.')

## トレーニングスクリプトを作成する

次の2つを実行して作成します:
1. 新しい実験用のフォルダー
2. **scikit-learn**を使用してモデルをトレーニングし、**matplotlib**を使用してROC曲線をプロットするトレーニングスクリプトファイル.

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Set regularization hyperparameter (passed as an argument to the script)
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 環境を定義する

Azure Machine Learningで実験としてPythonスクリプトを実行すると、スクリプトの実行コンテキストを定義するConda環境が作成されます。 Azure Machine Learningは、多くの一般的なパッケージを含むデフォルト環境を提供します。実験の実行に必要なライブラリを含む**azureml-defaults**パッケージ、および**pandas**や**numpy**などの一般的なパッケージが含まれます。

また、**conda**または**pip**を使用して独自の環境を定義し、パッケージを追加して、実験で必要なすべてのライブラリにアクセスできるようにすることもできます。 

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create a Python environment for the experiment
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
diabetes_env.docker.enabled = True # Use a docker container

# Create a set of package dependencies (conda or pip as required)
diabetes_packages = CondaDependencies.create(conda_packages=['pip==19.3.1','scikit-learn','ipykernel','matplotlib', 'pandas'],
                                          pip_packages=['azureml-sdk','pyarrow'])

# Add the dependencies to the environment
diabetes_env.python.conda_dependencies = diabetes_packages

print(diabetes_env.name, 'defined.')

これで、Estimator（またはRunConfig）に割り当てることで、実験に環境を使用できます。

次のコードは、作成した環境を汎用推定器に割り当て、実験を送信します。実験の実行中に、ウィジェットおよび**azureml_logs/60_control_log.txt**出力ログで実行の詳細を確認すると、conda環境が構築されていることがわかります。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Set the script parameters
script_params = {
    '--regularization': 0.1
}

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
              script_params=script_params,
              compute_target = 'local',
              environment_definition = diabetes_env,
              entry_script='diabetes_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

実験では、必要なすべてのパッケージを含む環境が正常に使用されました。AzureMachine Learning Studioで実験を実行するか、**scikit-learn**を使用して訓練されたモデルを含む以下のコードを実行することで、メトリックと出力を表示できます。および**matplotlib**を使用して生成されたROCチャート画像。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 環境を登録する

必要なパッケージを使用して環境を定義する手間がかかったら、ワークスペースに登録できます。

In [ ]:
# Register the environment
diabetes_env.register(workspace=ws)

環境は、最初に作成したときに割り当てた名前で登録されることに注意してください（この場合、*diabetes-experiment-env*）。

環境を登録すると、同じ要件を持つスクリプトに再利用できます。たとえば、別のアルゴリズムを使用して糖尿病モデルをトレーニングするためのフォルダーとスクリプトを作成しましょう:

In [ ]:
import os

# Create a folder for the experiment files
experiment_folder = 'diabetes_training_tree'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get the experiment run context
run = Run.get_context()

# load the diabetes data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a decision tree model
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

これで、登録済みの環境を取得し、それを使用して代替トレーニングスクリプトを実行する新しい実験の推定器を構成できます（決定ツリー分類子にはハイパーパラメーター値が必要ないため、今回はスクリプトパラメーターはありません）。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails

registered_env = Environment.get(ws, 'diabetes-experiment-env')

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator
estimator = Estimator(source_directory=experiment_folder,
              inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
              compute_target = 'local',
              environment_definition = registered_env,
              entry_script='diabetes_training.py')

# Create an experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

# Run the experiment
run = experiment.submit(config=estimator)
# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

今回は、前回の実行から一致する環境がキャッシュされているため、実験がより迅速に実行されます。したがって、ローカルコンピューティングで再作成する必要はありません。ただし、異なるコンピューティングターゲット上でも同じ環境が作成および使用され、実験スクリプトの実行コンテキストの一貫性が確保されます。

実験のメトリックと出力を見てみましょう。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

このモデルはロジスティック回帰モデルよりもわずかに優れているように見えるので、登録しましょう。

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator + Environment (Decision Tree)'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 登録済み環境を表示する

独自の環境を登録することに加えて、一般的な実験タイプに事前に構築された「キュレートされた」環境を活用できます。次のコードは、登録されているすべての環境をリストします:

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    print("Name",env)

すべてのキュレーションされた環境には、**AzureML-**で始まる名前が付いています（独自の環境にこのプレフィックスを使用することはできません）。

キュレーションされた環境をさらに詳しく調べて、それぞれに含まれるパッケージを確認しましょう。

In [ ]:
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

> **詳しくは**: Azure Machine Learningの環境の詳細については、[Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments)を参照してください